In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../data/processed/df_res.csv')

In [3]:
print(df.shape)
df.info()

(38850, 46)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38850 entries, 0 to 38849
Data columns (total 46 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   game_id                            38850 non-null  int64  
 1   competition_id                     38850 non-null  float64
 2   season                             38850 non-null  float64
 3   round                              38850 non-null  float64
 4   date                               38850 non-null  object 
 5   home_club_id                       38850 non-null  float64
 6   away_club_id                       38850 non-null  float64
 7   home_club_goals                    38850 non-null  float64
 8   away_club_goals                    38850 non-null  float64
 9   home_club_position                 38850 non-null  float64
 10  away_club_position                 38850 non-null  float64
 11  home_club_manager_name             38850 n

In [4]:
df.columns

Index(['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id',
       'away_club_id', 'home_club_goals', 'away_club_goals',
       'home_club_position', 'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee',
       'home_club_formation', 'away_club_formation',
       'domestic_competition_id_home_club', 'squad_size_home_club',
       'average_age_home_club', 'foreigners_number_home_club',
       'national_team_players_home_club', 'stadium_name_home_club',
       'stadium_seats_home_club', 'net_transfer_record_home_club',
       'last_season_home_club', 'domestic_competition_id_away_club',
       'squad_size_away_club', 'average_age_away_club',
       'foreigners_number_away_club', 'national_team_players_away_club',
       'stadium_name_away_club', 'stadium_seats_away_club',
       'net_transfer_record_away_club', 'last_season_away_club', 'country_id',
       'is_major_national_league', 'year', 'month', 'day',

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [6]:
# Припускаємо, що ваш підготовлений DataFrame називається df
# Переконайтеся, що ваш DataFrame df вже завантажено та оброблено,
# і він має структуру, яку ви описали.

# Визначення ознак (X) та цільових змінних (y)
# Виключаємо game_id, дату та оригінальні object колонки,
# якщо їх закодовані числові версії присутні
X = df.drop(['game_id', 'date', 'home_club_goals', 'away_club_goals',
             'home_club_formation', 'away_club_formation',
             'net_transfer_record_home_club', 'net_transfer_record_away_club'], axis=1)

y = df[['home_club_goals', 'away_club_goals']] # Цільові змінні

# Перевірка, чи залишилися в X нечислові колонки (крім boolean, які RandomForest може обробляти)
# Якщо є object колонки, які не були закодовані, їх потрібно обробити.
non_numeric_cols = X.select_dtypes(include=['object']).columns
if len(non_numeric_cols) > 0:
    print(f"Увага: У наборі ознак X залишилися нечислові колонки: {list(non_numeric_cols)}")
    print("RandomForestRegressor не може працювати безпосередньо з object Dtype.")
    print("Будь ласка, закодуйте ці колонки (наприклад, One-Hot Encoding) або видаліть їх.")
    # Якщо ви впевнені, що ці колонки не потрібні або були закодовані в інших колонках:
    # X = X.drop(non_numeric_cols, axis=1)

In [7]:
# Перетворення boolean колонки на числову (хоча RandomForest може обробляти bool, явне перетворення не завадить)
if 'is_major_national_league' in X.columns:
    X['is_major_national_league'] = X['is_major_national_league'].astype(int)

In [8]:
# Розбиття даних на навчальну та тестову вибірки
# test_size=0.2 означає, що 20% даних піде на тестування, 80% - на навчання
# random_state для відтворюваності результатів
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nРозмір навчальної вибірки: {X_train.shape[0]} рядків")
print(f"Розмір тестової вибірки: {X_test.shape[0]} рядків")
print(f"Кількість ознак: {X_train.shape[1]}")
print(f"Кількість цільових змінних: {y_train.shape[1]}")


Розмір навчальної вибірки: 31080 рядків
Розмір тестової вибірки: 7770 рядків
Кількість ознак: 38
Кількість цільових змінних: 2


In [9]:
# --- Створення та навчання моделі RandomForestRegressor ---
# n_estimators - кількість дерев у лісі (можна налаштовувати)
# random_state для відтворюваності
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1) # n_jobs=-1 використовує всі доступні ядра процесора

print("\nНавчання моделі RandomForestRegressor...")
model.fit(X_train, y_train)
print("Навчання завершено.")


Навчання моделі RandomForestRegressor...
Навчання завершено.


In [10]:
# --- Здійснення передбачень на тестовій вибірці ---
y_pred = model.predict(X_test)

In [11]:
# Оскільки передбачення можуть бути дробовими, а голи - цілими числами,
# часто округлюють передбачення до найближчого цілого невід'ємного числа.
y_pred_rounded = np.maximum(0, np.round(y_pred)).astype(int)

In [12]:
# --- Оцінка якості моделі ---
print("\nОцінка моделі на тестовій вибірці:")

# Оцінка для передбачення home_club_goals
print("\nДля передбачення home_club_goals:")
mae_home = mean_absolute_error(y_test['home_club_goals'], y_pred[:, 0])
mse_home = mean_squared_error(y_test['home_club_goals'], y_pred[:, 0])
rmse_home = np.sqrt(mse_home)
r2_home = r2_score(y_test['home_club_goals'], y_pred[:, 0])

print(f"  Mean Absolute Error (MAE): {mae_home:.4f}")
print(f"  Mean Squared Error (MSE): {mse_home:.4f}")
print(f"  Root Mean Squared Error (RMSE): {rmse_home:.4f}")
print(f"  R-squared (R2): {r2_home:.4f}")

# Оцінка для передбачення away_club_goals
print("\nДля передбачення away_club_goals:")
mae_away = mean_absolute_error(y_test['away_club_goals'], y_pred[:, 1])
mse_away = mean_squared_error(y_test['away_club_goals'], y_pred[:, 1])
rmse_away = np.sqrt(mse_away)
r2_away = r2_score(y_test['away_club_goals'], y_pred[:, 1])

print(f"  Mean Absolute Error (MAE): {mae_away:.4f}")
print(f"  Mean Squared Error (MSE): {mse_away:.4f}")
print(f"  Root Mean Squared Error (RMSE): {rmse_away:.4f}")
print(f"  R-squared (R2): {r2_away:.4f}")

# Оцінка округлених передбачень (якщо потрібно оцінювати саме цілі числа)
print("\nОцінка округлених передбачень (цілі числа):")
mae_home_rounded = mean_absolute_error(y_test['home_club_goals'], y_pred_rounded[:, 0])
mae_away_rounded = mean_absolute_error(y_test['away_club_goals'], y_pred_rounded[:, 1])

print(f"  Середня абсолютна похибка (MAE) home_club_goals (округлені): {mae_home_rounded:.4f}")
print(f"  Середня абсолютна похибка (MAE) away_club_goals (округлені): {mae_away_rounded:.4f}")


# --- Приклад використання навченої моделі для передбачення для нового матчу ---
# Створіть новий DataFrame з ознаками для матчу, який хочете передбачити.
# Переконайтеся, що колонки розташовані в тому ж порядку, що й у X_train.
# new_match_features = pd.DataFrame([...], columns=X_train.columns)
# predicted_goals = model.predict(new_match_features)
# print("\nПередбачені голи для нового матчу:", np.maximum(0, np.round(predicted_goals)).astype(int))


Оцінка моделі на тестовій вибірці:

Для передбачення home_club_goals:
  Mean Absolute Error (MAE): 0.9210
  Mean Squared Error (MSE): 1.3881
  Root Mean Squared Error (RMSE): 1.1782
  R-squared (R2): 0.1894

Для передбачення away_club_goals:
  Mean Absolute Error (MAE): 0.8327
  Mean Squared Error (MSE): 1.1247
  Root Mean Squared Error (RMSE): 1.0605
  R-squared (R2): 0.1659

Оцінка округлених передбачень (цілі числа):
  Середня абсолютна похибка (MAE) home_club_goals (округлені): 0.8965
  Середня абсолютна похибка (MAE) away_club_goals (округлені): 0.8040


In [13]:
# Припускаємо, що у вас вже є y_test (фактичні значення)
# та y_pred_rounded (округлені передбачення моделі)

# Точність передбачення home_club_goals
exact_accuracy_home = np.mean(y_test['home_club_goals'] == y_pred_rounded[:, 0])
print(f"Абсолютна точність передбачення home_club_goals: {exact_accuracy_home:.4f}")

# Точність передбачення away_club_goals
exact_accuracy_away = np.mean(y_test['away_club_goals'] == y_pred_rounded[:, 1])
print(f"Абсолютна точність передбачення away_club_goals: {exact_accuracy_away:.4f}")

# Точність, коли передбачення обох команд EXACTLY співпали з фактичними
exact_accuracy_both = np.mean((y_test['home_club_goals'] == y_pred_rounded[:, 0]) &
                              (y_test['away_club_goals'] == y_pred_rounded[:, 1]))
print(f"Абсолютна точність передбачення обох (home та away) голів: {exact_accuracy_both:.4f}")

Абсолютна точність передбачення home_club_goals: 0.3306
Абсолютна точність передбачення away_club_goals: 0.3533
Абсолютна точність передбачення обох (home та away) голів: 0.1283


In [14]:
# Припускаємо, що у вас вже є y_test (фактичні значення)
# та y_pred_rounded (округлені передбачення моделі)

# Визначення фактичного результату матчу
# 1: Home Win, 0: Draw, -1: Away Win
actual_outcome = np.sign(y_test['home_club_goals'] - y_test['away_club_goals'])

# Визначення передбаченого результату матчу на основі округлених передбачень
predicted_outcome = np.sign(y_pred_rounded[:, 0] - y_pred_rounded[:, 1])

# Розрахунок точності (відсоток правильних передбачень результату)
match_outcome_accuracy = np.mean(actual_outcome == predicted_outcome)

print(f"\nТочність передбачення результату матчу (Перемога/Нічия/Поразка): {match_outcome_accuracy:.4f}")


Точність передбачення результату матчу (Перемога/Нічия/Поразка): 0.5625


In [15]:
import joblib

# Зберегти модель у файл
# joblib.dump(model, '../res/random_forest_models/v0/random_forest_model.pkl')

In [18]:
X.head()

,competition_id,season,round,home_club_id,away_club_id,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,stadium,...,country_id,is_major_national_league,year,month,day,dayofweek,home_club_formation_code,away_club_formation_code,net_transfer_record_home_club_num,net_transfer_record_away_club_num
0,0.538462,0.0,0.000000,0.000272,0.000345,0.368421,0.421053,0.410874,0.898639,0.930612,...,0.122807,1,0.0,0.636364,0.333333,1.000000,0.00000,0.000000,0.692991,0.645976
1,0.538462,0.0,0.000000,0.000181,0.000753,0.631579,0.315789,0.909842,0.772109,0.214286,...,0.122807,1,0.0,0.636364,0.300000,0.833333,0.03125,0.176471,0.665446,0.660023
2,0.538462,0.0,0.297297,0.000118,0.000181,0.000000,0.736842,0.472815,0.910204,0.700000,...,0.122807,1,0.0,0.636364,0.566667,1.000000,0.00000,0.029412,0.575022,0.665446
3,0.538462,0.0,0.594595,0.000181,0.000190,0.894737,0.526316,0.909842,0.080952,0.214286,...,0.122807,1,0.0,0.636364,0.800000,1.000000,0.03125,0.000000,0.665446,0.802279
4,0.538462,0.0,0.891892,0.000118,0.000345,0.000000,0.736842,0.472815,0.898639,0.700000,...,0.122807,1,0.0,0.727273,0.433333,0.833333,0.00000,0.441176,0.575022,0.645976
